## Ejercicio

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.


# Desarrollo

## Librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import unicodedata

import multiprocessing
from gensim.models import Word2Vec

In [2]:
# Posibles bandas
os.listdir("./libro_dataset")

['Alejandro Dumas - El Conde de Montecristo.pdf',
 'El_Conde_de_Montecristo.txt',
 'Sun_Tzu_El_Arte_de_la_Guerra.txt',
 'Alejandro Dumas - Los Tres Mosqueteros.pdf',
 'Sun Tzu - El Arte de la Guerra.pdf',
 'Los_Tres_Mosqueteros.txt',
 'Julio Verne - La Vuelta al Mundo en 80 dias.pdf',
 'Julio_Verne_La_Vuelta_al_Mundo_en_80_dias.txt']

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
#df = pd.read_csv('libro_dataset/Julio_Verne_La_Vuelta_al_Mundo_en_80_dias.txt', sep='/n', header=None, names=['texto'])
df = pd.read_csv('libro_dataset/Los_Tres_Mosqueteros.txt', sep='/n', header=None, names=['texto'])
df.head()

/var/folders/sp/hjjl899x4js87jvp7rfdpj2w0000gn/T/ipykernel_26458/1763536272.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('libro_dataset/Los_Tres_Mosqueteros.txt', sep='/n', header=None, names=['texto'])


,texto
0,Los Tres Mosqueteros
1,Alejandro Dumas
2,textos.info
3,Biblioteca digital abierta
4,1


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 22743


#  Preprocesamiento

In [5]:
# Pasar todo a minúsculas
df['texto'] = df['texto'].str.lower()
# Eliminar puntuación
df['texto'] = df['texto'].str.replace(r'[^\w\s]', '', regex=True)
# Sacar numeros? Me parece que ayuda sacar los numeros ya estan los numeros de las 
# paginas incluidos lo cual agrega ruido
#df['texto'] = df['texto'].str.replace(r'\d+', '', regex=True)

In [6]:
# Función para quitar acentos de una cadena
def quitar_acentos(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize('NFKD', texto)
        texto = texto.encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Aplicar la función a la columna de texto
df['texto'] = df['texto'].apply(quitar_acentos)

In [7]:
# Importar librerías
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stopwords en español
#stopwords_es = set(stopwords.words('spanish'))
stopwords_es = set([
    'el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas',
    'de', 'del', 'al', 'a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'desde', 'en', 'entre', 'hacia', 'hasta', 'para', 'por', 'según', 'sin', 'sobre', 'tras',
    'y', 'e', 'ni', 'o', 'u', 'pero', 'sino', 'aunque', 'mientras', 'como', 'cuando', 'donde', 'si', 'porque', 'pues', 'que'
])



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastiancarreras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 22743


In [9]:
#!pip install --upgrade tensorflow
#from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
for _, row in df[:None].iterrows():
    tokens = text_to_word_sequence(row[0])
    tokens_filtrados = [t for t in tokens if t not in stopwords_es] #Eliminar stopwords 
    sentence_tokens.append(tokens)

/var/folders/sp/hjjl899x4js87jvp7rfdpj2w0000gn/T/ipykernel_26458/2536677296.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tokens = text_to_word_sequence(row[0])


In [10]:
# Damos un vistazo al libro
sentence_tokens[:25]

[['los', 'tres', 'mosqueteros'],
 ['alejandro', 'dumas'],
 ['textosinfo'],
 ['biblioteca', 'digital', 'abierta'],
 ['1'],
 ['texto', 'num', '172'],
 ['titulo', 'los', 'tres', 'mosqueteros'],
 ['autor', 'alejandro', 'dumas'],
 ['etiquetas', 'novela'],
 ['editor', 'edu', 'robsy'],
 ['fecha', 'de', 'creacion', '16', 'de', 'mayo', 'de', '2016'],
 ['fecha', 'de', 'modificacion', '16', 'de', 'mayo', 'de', '2016'],
 ['edita', 'textosinfo'],
 ['maison', 'carree'],
 ['c', 'ramal', '48'],
 ['07730', 'alayor', 'menorca'],
 ['islas', 'baleares'],
 ['espana'],
 ['mas', 'textos', 'disponibles', 'en', 'httpwwwtextosinfo'],
 ['2'],
 ['prefacio'],
 ['en',
  'el',
  'que',
  'se',
  'race',
  'constar',
  'que',
  'pese',
  'a',
  'sus',
  'nombres',
  'en'],
 ['os',
  'y',
  'en',
  'is',
  'los',
  'heroes',
  'de',
  'la',
  'historia',
  'que',
  'vamos',
  'a'],
 ['tener',
  'el',
  'honor',
  'de',
  'contar',
  'a',
  'nuestros',
  'lectores',
  'no',
  'tienen'],
 ['nada', 'de', 'mitologico']]

## Callback to track loss

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
multiprocessing.cpu_count()

12

## Modelo y parametros

In [13]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(
                min_count=10,       # numero de veces que aparece la palabra
                window=3,          # más contexto puede ayudar
                vector_size=100,   # uso un valor chico para un corpus chico/Aumentar para embeddings más ricos
                workers=multiprocessing.cpu_count(), # usar todos los cores
                negative=18,       # suficiente para un corpus narrativo
                sg=1               # skipgram capta mejor significados en corpus pequeños
                ) 

In [14]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
# base porque anda dentrod e todo bien
""" w2v_model = Word2Vec(
                min_count=10,       # numero de veces que aparece la palabra
                window=3,          # más contexto puede ayudar
                vector_size=100,   # uso un valor chico para un corpus chico/Aumentar para embeddings más ricos
                workers=multiprocessing.cpu_count(), # usar todos los cores
                negative=20,       # suficiente para un corpus narrativo
                sg=1               # skipgram capta mejor significados en corpus pequeños
                ) """ 

' w2v_model = Word2Vec(\n                min_count=10,       # numero de veces que aparece la palabra\n                window=3,          # más contexto puede ayudar\n                vector_size=100,   # uso un valor chico para un corpus chico/Aumentar para embeddings más ricos\n                workers=multiprocessing.cpu_count(), # usar todos los cores\n                negative=20,       # suficiente para un corpus narrativo\n                sg=1               # skipgram capta mejor significados en corpus pequeños\n                ) '

In [15]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [16]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 22743


In [17]:
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de palabras distintas en el corpus: 1942


# Entrenar embeddings

In [18]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 132668.578125
Loss after epoch 1: 123445.8125
Loss after epoch 2: 123257.234375
Loss after epoch 3: 119391.3125
Loss after epoch 4: 119207.1875
Loss after epoch 5: 117081.625
Loss after epoch 6: 114008.0
Loss after epoch 7: 115035.5625
Loss after epoch 8: 111928.1875
Loss after epoch 9: 106064.125
Loss after epoch 10: 104832.125
Loss after epoch 11: 104167.0
Loss after epoch 12: 100524.125
Loss after epoch 13: 103461.375
Loss after epoch 14: 99081.0
Loss after epoch 15: 103359.125
Loss after epoch 16: 101833.875
Loss after epoch 17: 99081.75
Loss after epoch 18: 100671.25
Loss after epoch 19: 91527.75
Loss after epoch 20: 88632.0
Loss after epoch 21: 90092.0
Loss after epoch 22: 92132.25
Loss after epoch 23: 91876.25
Loss after epoch 24: 88689.5
Loss after epoch 25: 89971.75
Loss after epoch 26: 82292.75
Loss after epoch 27: 91731.0
Loss after epoch 28: 89375.75
Loss after epoch 29: 90365.5


(3684366, 6561000)

In [19]:
## Análisis de Similitudes
# Palabras de interés
palabras_clave = ['rey', 'cardenal', 'mosqueteros', 'milady', 'reina']

# Calculamos similitudes
similitudes = {}
for palabra in palabras_clave:
    try:
        similitudes[palabra] = [item[0] for item in w2v_model.wv.most_similar(palabra, topn=5)]
    except KeyError:
        continue

print("Palabras más similares:")
for palabra, similares in similitudes.items():
    print(f"{palabra}: {', '.join(similares)}")


Palabras más similares:
rey: luis, xiii, escudero, arrestar, cardenal
cardenal: rey, escudero, suizo, comisario, dartagnan
mosqueteros: soldados, companeros, ayuntamiento, cirujano, ingleses
milady: prisionera, felton, burgues, baron, dartagnan
reina: novicia, madre, fiesta, baile, corte


In [20]:
## Tests de Analogías
analogias = [
    ('dartagnan', 'mosquetero', 'porthos'),       # Relación de pertenencia
    ('reina', 'ana', 'rey'),                      # Reina Ana, Rey Luis XIII
    ('mosqueteros', 'amistad', 'milady'),         # Contraste: mosqueteros/amistad vs. Milady/traición
    ('francia', 'paris', 'inglaterra'),           # País y capital
    ('buckingham', 'inglaterra', 'richelieu'),    # Buckingham:Inglaterra :: Richelieu:Francia
    ('milady', 'serpiente', 'athos'),             # Milady comparada con serpiente, Athos con león
    ('mosqueteros', 'espada', 'guardia'),         # Mosqueteros usan espada, Guardia usa lanza
    ('dartagnan', 'amor', 'constance'),           # d'Artagnan y su interés amoroso
    ('milady', 'venganza', 'winter'),             # Milady busca venganza, Lord de Winter es su objetivo
    ('cardenal', 'poder', 'rey'),                 # Cardenal Richelieu y el poder, Rey y la corona
    ('milady', 'traicion', 'constance'),          # Milady traiciona, Constanza es leal
    ('mosqueteros', 'uno', 'todos'),              # Uno para todos, todos para uno
]

print("Test de analagías:")
for a, b, c in analogias:
    try:
        resultado = w2v_model.wv.most_similar(positive=[b, c], negative=[a], topn=1)[0][0]
        print(f"{a} - {b} + {c} = {resultado}")
    except KeyError as e:
        print(f"Palabra no encontrada: {e}")

Test de analagías:
dartagnan - mosquetero + porthos = zafiro
reina - ana + rey = luis
mosqueteros - amistad + milady = muchacha
francia - paris + inglaterra = partir
buckingham - inglaterra + richelieu = xiii
milady - serpiente + athos = caja
mosqueteros - espada + guardia = esperanza
dartagnan - amor + constance = decidme
milady - venganza + winter = lord
cardenal - poder + rey = podian
milady - traicion + constance = callaos
mosqueteros - uno + todos = pobres


# Visualización

In [21]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [22]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [23]:
# Graficar los embedddings en 3D
vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab